In [1]:
from __future__ import division

In [2]:
import csv
import pandas as pd
import datetime
import string
import numpy as np
import helper as hp
import sq
import jp
import urllib

In [3]:
pd.set_option('display.max_columns', None)
Gb = 1000000000

In [4]:
#store = pd.HDFStore('access_log.h5')

In [5]:
def get_server_from_URL(URLs):
    result = []
    
    for u in URLs:
        if u.startswith('http://'):
            temp = u.split('http://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
        elif u.startswith('http://'):
            temp = u.split('https://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
    return result

In [6]:
def remove_safe_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True
                break
                
        if (m == False):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result            

In [7]:
def get_match_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True                
                break
                                
        if (m == True):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result   

In [8]:
#squid = sq.read_squid_log('access.log')

In [9]:
#squid.tail()

In [10]:
#safe_websites = 'google|microsoft|trendmicro|gstatic.com|bdpinsight.eu|.gov.hk'

In [11]:
#access = squid[squid['URL'].str.contains(safe_websites) == False]

In [ ]:
#sum(squid.bytes)/Gb

In [ ]:
URL = urllib.urlopen('https://doc-04-0c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/12ir2olou9sklqs463vdda1nv70kbpst/1443074400000/13625217592848274312/*/0B9C3OkBK2B-JRlBkYXI4UXo5RVU?e=download')

In [ ]:
juniper = jp.read_syslog_juniper(URL)

In [ ]:
juniper.head()